# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [ ]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

In [ ]:
!mv abq-food-inspections-fy2010.csv food.csv

In [ ]:
!csvcut -n food.csv

In [ ]:
!wc -l food.csv

In [ ]:
!csvstat food.csv


FACILITY_NAME is the name of the inspected place

FACILITY KEY is the unique key associated to the inspected place

Site_address is the street of the place

City is the city in which it is located

State is the state in which it is locate

Zip is the zip code in which the place is located

Owner_key is the unique id of the owner of the facility

Owner_name is the name of the owner of the place

Nature_of_business is the type of the the business

Street_number,name,type, post_directional gives the address of the facility

Phone is the phone number of the facility

Program category is the code of the permit of the facility 

Program category description is the description the permit of the facility 

Inspected date tells us when it was inspected

Serial number tells the ordered serial number of the inspection

Action_code tells us the internal code of the action which was taken

Action_Desc is the description of the associated action code

Result_code is the result of the inspection 

Result_desc describes the result

Violation_code is the internal code of the violation which happened 

Violation_desc desccibes the violation which happened

Inspection_memo is the notes made by the inspector during the inspection


The columns which highly interest me are zip codes, time stamp, inspection type ,action, violations.

There are missing values in Site address, nature of business, street number,street name,street type,post directional,phone,program category description,inspection type,violaition code,violation desc,inspection memo


Certain Questions:
1. Who is inspected the most?
2. What are the types of innspection or why are they inspected?
3. WHere are certain inspections occuring the most?
4.What is the time when these inspections are at the peak?
5. What are the major violations?

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

In [ ]:
%load_ext sql

In [ ]:
!echo 'redspot' | sudo -S service postgresql restart

In [ ]:
!createdb -U dbuser ex04

In [ ]:
%sql postgresql://dbuser@localhost:5432/ex04

In [ ]:
%%sql
DROP TABLE IF EXISTS food;
CREATE TABLE food (
    facility_name CHAR(77),
    facility_key INTEGER,
    site_address CHAR(35),
    city CHAR(15),
    state CHAR(2),
    zip_address CHAR(20),
    owner_key INTEGER,
    owner_name CHAR(82),
    business_type CHAR(25),
    street_number CHAR(10),
    steet_name CHAR(24),
    street_type CHAR(4),
    direction CHAR(2),
    phone BIGINT,
    program_category CHAR(4),
    program_des CHAR(40),
    inspection_date TIMESTAMP,
    inspection_type INTEGER,
    inspection_des CHAR(32),
    serial CHAR(9),
    action_code CHAR(2),
    action_des CHAR (34),
    result_code CHAR(2),
    result_des CHAR(37),
    violation_code char(10),
    violation_des CHAR(50),
    inspection_memo INTEGER
    )


In [ ]:
%%sql
COPY food FROM '/home/jovyan/work/food.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

In [ ]:
%%sql
SELECT count(*) FROM food

## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT facility_name,count(*) As count FROM food
    Group by facility_name
    )sub
ORDER BY count DESC
LIMIT 5

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT zip_address,count(*) As count FROM food
    Group by zip_address
    )sub
ORDER BY count DESC
LIMIT 5

These are the top 5 areas and facilities which are inspected

In [ ]:
%%sql

SELECT inspection_des,count(*) as count FROM food
group by inspection_des
order by count desc

In [ ]:
%%sql
select action_code FROM food
where inspection_des='EMERGENCY RESPONSE'

11% of all the inspections are emergency response.We go on to check further.However most of them has no action taken as the inspector could not check it indicating a possible lack of inspector. On this assumption going forward.

In [ ]:
%%sql 
CREATE TABLE T1 AS
(
select zip_address,inspection_date FROM food
where inspection_des='EMERGENCY RESPONSE'
)

In [ ]:
%%sql

select * from T1

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT zip_address,count(*) As count FROM T1
    Group by zip_address
    )sub
ORDER BY count DESC
LIMIT 5

In [ ]:
%%sql
SELECT sub.* FROM
 (
  SELECT inspection_date,count(*) As count FROM T1
    Group by inspection_date
    )sub
ORDER BY count DESC
LIMIT 5

The above shows thae areas which require deployment of inspectors and the month of september has the most demand of inspectors for emergency response indicating a high alert.

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table